- 방문 주차에 따른 코호트를 생성하고, 코호트 별로 weekly 리텐션을 구하고 이를 heatmap으로 시각화

> [코드 참고 링크](https://www.youtube.com/watch?v=WWUG7T9ixTs)

In [7]:
# 전처리한 데이터 불러오기
import pandas as pd

train_df = pd.read_csv('C://Users//and_c//Desktop//sparta_ga4//train_df3.csv', low_memory = False)
df = train_df.copy()

AttributeError: partially initialized module 'fsspec' has no attribute 'utils' (most likely due to a circular import)

In [4]:
df.info()

NameError: name 'df' is not defined

In [5]:
df.head()

NameError: name 'df' is not defined

In [ ]:
# 데이터프레임에 한 열의 값을 바꾸기 위해서 def함수를 만들고 apply 적용
import pandas as pd
import datetime #날짜와 시간을 다루는 라이브러리
import pytz #다양한 시간대를 처리하기 위해 사용하는 라이브러리

# Posix 시간을 utc 시간으로 변환하는 함수
def posix_to_utc(visitStartTime):
    utc_time = datetime.datetime.utcfromtimestamp(visitStartTime)
    return utc_time

# Posix 시간을 UTC 지정한 로컬 시간대로 변환하는 함수
def posix_to_utc_local(visitStartTime):
    utc_time = datetime.datetime.utcfromtimestamp(visitStartTime) #posix -> utc시간대로
    local_timezone = pytz.timezone('America/New_York')  # 지역 시간대
    return utc_time.astimezone(local_timezone) # 그 지역 시간대로 변환

In [ ]:
df['visitStartTime'] = df['visitStartTime'].apply(posix_to_utc)

In [ ]:
# date 칼럼 추가하기
df['date'] = df['visitStartTime'].dt.date

In [ ]:
# week기준으로 구분한 칼럼 추가
df['period_week'] = df['visitStartTime'].dt.to_period('W').apply(lambda x: x.start_time.strftime('%Y-%m-%d'))


In [ ]:
# user의 첫 방문날짜 칼럼 추가
df['cohort_week'] = df.groupby('fullVisitorId')['period_week'].transform('min')

In [ ]:
df[['period_week','cohort_week']].tail()

In [ ]:
df['period_week'] = pd.to_datetime(df['period_week'])
df['cohort_week'] = pd.to_datetime(df['cohort_week'])

In [ ]:
# Cohort_index 구하기 ...
df['cohort_index'] = (df['period_week'] - df['cohort_week'])/7

In [ ]:
# df.groupby(['period_week','cohort_index'])['fullVisitorId'].nunique()
cohort_table = df.groupby(['period_week','cohort_index'])['fullVisitorId'].apply(pd.Series.nunique).reset_index()

In [ ]:
# create Table
cohort_table_pivot = cohort_table.pivot(index = 'period_week', columns=['cohort_index'], values='fullVisitorId')

In [ ]:
div_cohort_pivot = cohort_table_pivot.divide(cohort_table_pivot.iloc[:,0],axis=0)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# index 수정
# div_cohort_pivot.index = div_cohort_pivot.index.strftime('%Y-%m')

# heatmap
plt.figure(figsize=(21,10))
sns.heatmap(div_cohort_pivot, annot = True, cmap='Blues',fmt = '.0%')

In [ ]:
# plt.rcParams['figure.figsize'] = [10,8]
# plt.pcolor(table)
# # 또는 sns.heatmap()
# # 피벗테이블의 df를 pandas를 이용해 표현도 가능 df.style.background_gradient
# plt.title('Weekly Retention', fontsize=20)
# plt.xlabel('week', fontsize=14)
# plt.ylabel('visit', fontsize=14)
# plt.colorbar()

- 시간별/일별 heatmap 시각화

In [ ]:
# 데이터 로드
# import pandas as pd
# pd.read_csv('~')
df2 = train_df.copy()

In [ ]:
# 요일 및 시간대 추출
df2['day_of_week'] = df2['utc_time'].dt.day_name()
df2['hour'] = df2['utc_time'].dt.hour

In [ ]:
# 요일/시간대별 사용자 수 계산
heatmap_data = df.groupby(['day_of_week', 'hour']).size().unstack(fill_value=0)

In [ ]:
# heatmap으로 시각화
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(25, 8))
sns.heatmap(heatmap_data, cmap='YlGnBu', annot=True, fmt='d')
plt.title('Users by Day of Week and Hour')
plt.xlabel('Hour')
plt.ylabel('Day of Week')
plt.show()

Carrying Capapcity